In [8]:
import itertools
from collections import defaultdict
import dimod
from dimod import BQM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neal import SimulatedAnnealingSampler
from dwave.system import DWaveSampler, LeapHybridSampler
from dwave.system import EmbeddingComposite
from IPython.display import display, HTML

In [100]:
def university_bqm(R, C, D, T, L, can_lead, aud_size, classes_amount, stud_inter, can_in, time_cost):
    bqm = BQM("BINARY")
    P1 = 7
    P2 = 7
    P3 = 7
    P4 = 7
    P5 = 2
    P6 = 1
    P7 = 2
    P8 = 2
    
    """
    #Add hard constraint (with slack variables)#1
    for r, d, t in itertools.product(R, D, T):
        c1 = [(f"x_{c}_{l}_{r}_{d}_{t}", 1) for c, l in itertools.product(C, L) if (can_lead[l][c] and can_in[r][c])]
        bqm.add_linear_inequality_constraint(
            c1, lagrange_multiplier=P1, ub=1, label=f"c1_c_l_{r}_{d}_{t}")
    
    """
    #Add hard constraint (with slack variables)#1
    for r, d, t in itertools.product(R, D, T):
        for c1, l1 in itertools.product(range(len(C)), range(len(L))):
            if can_lead[L[l1]][C[c1]] and can_in[r][C[c1]]:
                for c2, l2 in itertools.product(range(len(C)), range(len(L))):
                    if can_lead[L[l2]][C[c2]] and can_in[r][C[c2]] and (c2 > c1 or (c2 == c1 and l2 > l1)):
                        bqm.add_quadratic(f"x_{C[c1]}_{L[l1]}_{r}_{d}_{t}", f"x_{C[c2]}_{L[l2]}_{r}_{d}_{t}", P1)
    """
    
    #Add hard constraint (with slack variables)#2
    for l, d, t in itertools.product(L, D, T):
        c2 = [(f"x_{c}_{l}_{r}_{d}_{t}", 1) for c, r in itertools.product(C, R) if (can_lead[l][c] and can_in[r][c])]
        bqm.add_linear_inequality_constraint(
            c2, lagrange_multiplier=P2, ub=1, label=f"c2_c_{l}_r_{d}_{t}")
    """
    
    #Add hard constraint (with slack variables)#1
    for l, d, t in itertools.product(L, D, T):
        for c1, r1 in itertools.product(range(len(C)), range(len(R))):
            if can_lead[l][C[c1]] and can_in[R[r1]][C[c1]]:
                for c2, r2 in itertools.product(range(len(C)), range(len(R))):
                    if can_lead[l][C[c2]] and can_in[R[r2]][C[c2]] and (c2 > c1 or (c2 == c1 and r2 > r1)):
                        bqm.add_quadratic(f"x_{C[c1]}_{l}_{R[r1]}_{d}_{t}", f"x_{C[c2]}_{l}_{R[r2]}_{d}_{t}", P2)
    """
    
    #Add hard constraint (with slack variables)#3
    for c, d, t in itertools.product(C, D, T):
        c3 = [(f"x_{c}_{l}_{r}_{d}_{t}", 1) for l, r in itertools.product(L, R) if (can_lead[l][c] and can_in[r][c])]
        bqm.add_linear_inequality_constraint(
            c3, lagrange_multiplier=P3, ub=1, label=f"c3_{c}_l_r_{d}_{t}")
    """
    #Add hard constraint#3
    for c, d, t in itertools.product(C, D, T):
        for l1, r1 in itertools.product(range(len(L)), range(len(R))):
            if can_lead[L[l1]][c] and can_in[R[r1]][c]:
                for l2, r2 in itertools.product(range(len(L)), range(len(R))):
                    if can_lead[L[l2]][c] and can_in[R[r2]][c] and (l2 > l1 or (l2 == l1 and r2 > r1)):
                        bqm.add_quadratic(f"x_{c}_{L[l1]}_{R[r1]}_{d}_{t}", f"x_{c}_{L[l2]}_{R[r2]}_{d}_{t}", P3)
    
    #Add hard constraint #4
    for c in C:
        c4 = [(f"x_{c}_{l}_{r}_{d}_{t}", 1) for r, l, d, t in itertools.product(R, L, D, T) if (can_lead[l][c] and can_in[r][c])]
        bqm.add_linear_equality_constraint(c4, lagrange_multiplier=P4, constant=-classes_amount[c])
        
    #Add soft constraint #1
    for c, d in itertools.product(C, D):
        c5 = [(f"x_{c}_{l}_{r}_{d}_{t}", 1) for l, r, t in itertools.product(L, R, T) if (can_lead[l][c] and can_in[r][c])]
        bqm.add_linear_inequality_constraint(
            c5, lagrange_multiplier=P5, ub=2, label=f"c5_{c}_l_r_{d}_t")
        
    #Add soft constraint #2
    for l1, r1, d1, t1 in itertools.product(L, R, D, T):
        for l2, r2, d2, t2 in itertools.product(L, R, D, T):
            if can_lead[l1]["KvantuDatoriLekcija"] and can_in[r1]["KvantuDatoriLekcija"] \
                and can_lead[l2]["KvantuDatoriPrakse"] and can_in[r2]["KvantuDatoriPrakse"]:
                if d2 < d1 or (d2 == d1 and t2 < t1):
                    bqm.add_quadratic(f"x_KvantuDatoriLekcija_{l1}_{r1}_{d}_{t}", f"x_KvantuDatoriPrakse_{l2}_{r2}_{d}_{t}", P6)
                    
    #Add soft constraint #3
    for d, t in itertools.product(D, T):
        for c1, r1, l1 in itertools.product(C, R, L):
            if can_lead[l1][c1] and can_in[r1][c1]:
                for c2, r2, l2 in itertools.product(C, R, L):
                    if can_lead[l2][c2] and can_in[r2][c2]:
                        if not (l1 == l2 and c1 == c2 and r1 == r2):
                            bqm.add_quadratic(f"x_{c1}_{l1}_{r1}_{d}_{t}", f"x_{c2}_{l2}_{r2}_{d}_{t}", stud_inter[c1][c2])
                            
    #Add soft constraint #4
    for c, r, l, d, t in itertools.product(C, R, L, D, T):
        if can_lead[l][c] and can_in[r][c]:
            bqm.add_linear(f"x_{c}_{l}_{r}_{d}_{t}", time_cost[t])
    
    return bqm

In [51]:
def is_sample_feasible(sample, R, C, D, T, L, can_lead, aud_size, classes_amount, can_in):
    
    
    #Check hard constraint #1
    for r, d, t in itertools.product(R, D, T):
        if sum([sample[f"x_{c}_{l}_{r}_{d}_{t}"] for c, l in itertools.product(C, L) if can_lead[l][c] and can_in[r][c]] ) > 1:
            return False
    
    #Check hard constraint #2
    for l, d, t in itertools.product(L, D, T):
        if sum([sample[f"x_{c}_{l}_{r}_{d}_{t}"] for c, r in itertools.product(C, R) if can_lead[l][c] and can_in[r][c]]) > 1:
            return False   
        
    #Check hard constraint #3
    for c, d, t in itertools.product(C, D, T):
        if sum([sample[f"x_{c}_{l}_{r}_{d}_{t}"] for l, r in itertools.product(L, R) if can_lead[l][c] and can_in[r][c]]) > 1:
            return False     
    
    #Check hard constraint #4
    for c in C:
        if sum([sample[f"x_{c}_{l}_{r}_{d}_{t}"] for r, l, d, t \
                in itertools.product(R, L, D, T) if can_lead[l][c] and can_in[r][c]]) != classes_amount[c]:
            return False
        
    return True

In [52]:
def get_scheedule(sample, R, C, D, T, L, can_lead, aud_size, can_in):
    scheedule = []
    for t in T:
        sc = [t]
        for d in D:
            if sum(sample[f"x_{c}_{l}_{r}_{d}_{t}"] for l, r, c in itertools.product(L, R, C) if can_lead[l][c] and can_in[r][c]) == 0:
                sc.append("-")
            else:
                val = ""
                for l, r, c in itertools.product(L, R, C):
                    if can_lead[l][c] and can_in[r][c]:
                        if sample[f"x_{c}_{l}_{r}_{d}_{t}"] == 1:
                            if val == "":
                                val = f"{c}, {l} ({r})"
                            else:
                                val = val + f"\n{c}, {l} ({r})"
                sc.append(val)
        scheedule.append(sc)
    display(HTML(pd.DataFrame(scheedule, columns = ["Time"] + D).to_html().replace("\\n","<br>")))

In [116]:
def prepare_big_test():
    


In [216]:
def prepare_small_test():
    


In [106]:
def prepare_medium_test():
    
    R = ["13", "14", "16", "345"]
    C = ["KvantuDatoriLekcija", "KvantuDatoriPrakse", "Kombinatorika","Kriptografija","GrafuTeorija"]
    D = ["P", "O"]
    T = [830, 1030, 1230, 1430, 1630, 1815, 2000]
    L = ["Kalnins","Ozolins","Liepa"]
    S = ["Jelisejevs", "Zajakins", "Pavlenko", "Gzibovska", "Kozjutinskis", "Kalnins"]
    
    hc_je = {"KvantuDatoriLekcija": True, "KvantuDatoriPrakse": True, "Kombinatorika": True,
             "Kriptografija": False, "GrafuTeorija": True}
    hc_za = {"KvantuDatoriLekcija": True, "KvantuDatoriPrakse": True, "Kombinatorika": True,
             "Kriptografija": False, "GrafuTeorija": True}
    hc_pa = {"KvantuDatoriLekcija": True, "KvantuDatoriPrakse": True, "Kombinatorika": True,
             "Kriptografija": False, "GrafuTeorija": True}
    hc_gz = {"KvantuDatoriLekcija": True, "KvantuDatoriPrakse": True, "Kombinatorika": True,
             "Kriptografija": True, "GrafuTeorija": False}
    hc_ko = {"KvantuDatoriLekcija": True, "KvantuDatoriPrakse": True, "Kombinatorika": True,
             "Kriptografija": True, "GrafuTeorija": False}
    hc_ka = {"KvantuDatoriLekcija": True, "KvantuDatoriPrakse": True, "Kombinatorika": True,
             "Kriptografija": True, "GrafuTeorija": False}
    has_course = {"Jelisejevs": hc_je, "Zajakins": hc_za, "Pavlenko": hc_pa,
                  "Gzibovska": hc_gz, "Kozjutinskis":hc_ko, "Kalnins": hc_ka}
    
    stud_inter = dict.fromkeys(C, {})
    for c in C:
        stud_inter[c] = dict.fromkeys(C, 0)
        
    for c1, c2 in itertools.product(C, C):
        if c1 != c2:
            stud_inter[c1][c2] = sum([1 for s in S if has_course[s][c1] and has_course[s][c2]])
            
    
    cl_ka = {"KvantuDatoriLekcija": False, "KvantuDatoriPrakse": True, "Kombinatorika": False,
             "Kriptografija": True, "GrafuTeorija": False}
    cl_oz = {"KvantuDatoriLekcija": True, "KvantuDatoriPrakse": False, "Kombinatorika": True, 
             "Kriptografija": False, "GrafuTeorija": False}
    cl_li = {"KvantuDatoriLekcija": False, "KvantuDatoriPrakse": False, "Kombinatorika": False, 
             "Kriptografija": True, "GrafuTeorija": True}
    can_lead = {"Kalnins": cl_ka, "Ozolins": cl_oz, "Liepa": cl_li}
    
    can_in = dict.fromkeys(R, {})
    for r in R:
        can_in[r] = dict.fromkeys(C, False)
        
    for r, c in itertools.product(R, C):
        if sum([1 for s in S if has_course[s][c]]) <= aud_size[r]:
            can_in[r][c] = True
        
    classes_amount = {"KvantuDatoriLekcija": 1, "KvantuDatoriPrakse": 1, "Kombinatorika": 2,
                      "Kriptografija": 2, "GrafuTeorija": 4}
    
    time_cost = {830: 0, 1030: 0, 1230: 0, 1430: 0, 1630: 0, 1815: 1, 2000: 2}
    
    return R, C, D, T, L, can_lead, aud_size, classes_amount, stud_inter, can_in, time_cost

In [107]:
#R, C, D, T, L, can_lead, aud_size, classes_amount, stud_inter, can_in, time_cost = prepare_big_test()
R, C, D, T, L, can_lead, aud_size, classes_amount, stud_inter, can_in, time_cost = prepare_medium_test()
#R, C, D, T, L, can_lead, aud_size, classes_amount, stud_inter, can_in, time_cost = prepare_small_test()
    
bqm = university_bqm(R, C, D, T, L, can_lead, aud_size, classes_amount, stud_inter, can_in, time_cost)
len(bqm.variables)

314

## Simulated Annealing

Can handle problem instances of any size

In [108]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads = 200)

print(sampleset)

    ... x_KvantuDatoriPrakse_Kalnins_16_P_830 energy num_oc.
9   ...                                     0    0.0       1
24  ...                                     0    0.0       1
38  ...                                     0    0.0       1
68  ...                                     0    0.0       1
78  ...                                     0    0.0       1
80  ...                                     0    0.0       1
91  ...                                     0    0.0       1
93  ...                                     0    0.0       1
97  ...                                     0    0.0       1
103 ...                                     0    0.0       1
153 ...                                     1    0.0       1
159 ...                                     1    0.0       1
2   ...                                     0    1.0       1
12  ...                                     0    1.0       1
14  ...                                     0    1.0       1
16  ...                 

In [109]:
best_sample = sampleset.first.sample
is_feasible = is_sample_feasible(best_sample, R, C, D, T, L, can_lead, aud_size, classes_amount, can_in)
print(is_feasible)
get_scheedule(best_sample, R, C, D, T, L, can_lead, aud_size, can_in)

True


,Time,P,O
0,830,"KvantuDatoriPrakse, Kalnins (16)",-
1,1030,"GrafuTeorija, Liepa (345)","GrafuTeorija, Liepa (16)"
2,1230,"Kriptografija, Kalnins (14)","KvantuDatoriLekcija, Ozolins (16)"
3,1430,"Kombinatorika, Ozolins (14)","Kriptografija, Kalnins (345)GrafuTeorija, Liepa (14)"
4,1630,"GrafuTeorija, Liepa (16)","Kombinatorika, Ozolins (13)"
5,1815,-,-
6,2000,-,-


## Quantum Annealing
First of all we create quantum and hybid solvers, by providing Leap subscription token

In [110]:
qpu_advantage = EmbeddingComposite(DWaveSampler(solver={'topology__type': 'pegasus'}, token="DEV-fc61310ff217929feac5a5ec87b1b5e23302c79a"))
hybrid_bqm_sampler = LeapHybridSampler(token="DEV-fc61310ff217929feac5a5ec87b1b5e23302c79a")

### Hybrid solver
Can be used for problem instances of any size

In [111]:
hybrid_sampleset = hybrid_bqm_sampler.sample(bqm)
print(hybrid_sampleset)

  ... x_KvantuDatoriPrakse_Kalnins_16_P_830 energy num_oc.
0 ...                                     0    0.0       1
['BINARY', 1 rows, 1 samples, 314 variables]


In [112]:
best_hybrid_sample = hybrid_sampleset.first.sample
is_feasible = is_sample_feasible(best_hybrid_sample, R, C, D, T, L, can_lead, aud_size, classes_amount, can_in)
print(is_feasible)
get_scheedule(best_hybrid_sample, R, C, D, T, L, can_lead, aud_size, can_in)

True


,Time,P,O
0,830,"KvantuDatoriLekcija, Ozolins (13)","GrafuTeorija, Liepa (13)"
1,1030,"GrafuTeorija, Liepa (14)","Kriptografija, Kalnins (345)GrafuTeorija, Liepa (14)"
2,1230,"KvantuDatoriPrakse, Kalnins (13)","Kombinatorika, Ozolins (14)"
3,1430,"GrafuTeorija, Liepa (13)","Kriptografija, Kalnins (16)"
4,1630,"Kombinatorika, Ozolins (13)",-
5,1815,-,-
6,2000,-,-


### Quantum solver
Can be used only for small problem instances with size of ~100 variables 

In [227]:
quantum_sampleset = qpu_advantage.sample(bqm, num_reads = 2000)
print(quantum_sampleset)

     slack_c7_LatVal_11a_O_t_0 ... x_pus_11b_P_955 energy num_oc. chain_b.
95                           1 ...               0    9.0       1 0.019231
472                          0 ...               0   12.0       1 0.019231
364                          1 ...               0   15.0       1 0.038462
127                          1 ...               0   16.0       1 0.028846
483                          0 ...               0   16.0       1 0.019231
596                          0 ...               0   18.0       1 0.028846
175                          1 ...               0   19.0       1 0.019231
434                          1 ...               0   19.0       1 0.009615
197                          0 ...               0   20.0       1 0.009615
250                          1 ...               1   20.0       1      0.0
229                          1 ...               1   21.0       1 0.038462
272                          1 ...               0   21.0       1      0.0
891                      

In [228]:
best_quantum_sample = quantum_sampleset.first.sample
is_feasible = is_sample_feasible(best_quantum_sample, R, C, D, T, L, can_lead, aud_size, classes_amount, can_in)
print(is_feasible)
get_scheedule(best_hybrid_sample, R, C, D, T, L, can_lead, aud_size, can_in)

True
11a
   Time           P           O
0   815  Matematika   Pusdienas
1   900      Sports  Matematika
2   955  Matematika      Sports
3  1040   Pusdienas      LatVal
4  1145           -           -
-----------------------------------------------
11b
   Time          P           O
0   815  Pusdienas  Matematika
1   900     LatVal   Pusdienas
2   955     Sports      LatVal
3  1040          -      Sports
4  1145          -           -
-----------------------------------------------
